In [49]:
#####All Packages#####

import math
import numpy as np,numpy.linalg
import pandas as pd
from scipy.linalg import cholesky
import random
from joblib import Parallel, delayed
from sklearn.cluster import KMeans
from sklearn.cluster import OPTICS
from functools import partial
from fancyimpute import MatrixFactorization
from scipy.stats import truncnorm
import csv

#####All functions#####

##to get a positive semidefinite correlation matrix
def _getAplus(A):
    eigval, eigvec = np.linalg.eig(A)
    Q = np.matrix(eigvec)
    xdiag = np.matrix(np.diag(np.maximum(eigval, 0)))
    return Q*xdiag*Q.T

def _getPs(A, W=None):
    W05 = np.matrix(W**.5)
    return  W05.I * _getAplus(W05 * A * W05) * W05.I

def _getPu(A, W=None):
    Aret = np.array(A.copy())
    Aret[W > 0] = np.array(W)[W > 0]
    return np.matrix(Aret)

def nearPD(A, nit=10):
    n = A.shape[0]
    W = np.identity(n) 
# W is the matrix used for the norm (assumed to be Identity matrix here)
    deltaS = 0
    Yk = A.copy()
    for k in range(nit):
        Rk = Yk - deltaS
        Xk = _getPs(Rk, W=W)
        deltaS = Xk - Rk
        Yk = _getPu(Xk, W=W)
    return Yk


##Computing truncated gaussian numbers
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)



In [42]:
####Simulation of the dataset####
##procedure for normal distributed values##

seed = 30
sample_size = 5000


#A1 is for the dispersion of the distribution (from low to high)
for A1 in range(0,3):

    #C1 is for the correlation (from low to high)
    for C1 in range(0,3):

        #for each city one separate sample
        for B1 in range(0,4):

            #first step: generate gaussian random values with borders
            X1 = get_truncated_normal(mean=2, sd=1*(1+A1), low=1, upp=3)
            #second step: scale the random values to mean=0, std=1 in order to apply the correlation matrix
            np.random.seed(seed+A1+C1+B1+1)
            rnd1 = (X1.rvs(sample_size)-np.mean(X1.rvs(sample_size)))/np.std(X1.rvs(sample_size))
            X2 = get_truncated_normal(mean=4, sd=2*(1+A1), low=3, upp=5)
            np.random.seed(seed+A1+C1+B1+2)
            rnd2 = (X2.rvs(sample_size)-np.mean(X2.rvs(sample_size)))/np.std(X2.rvs(sample_size))
            X3 = get_truncated_normal(mean=1500-B1*125, sd=(750-B1*62.5)*(1+A1), low=500, upp=2500-B1*250)
            np.random.seed(seed+A1+C1+B1+3)
            rnd3 = (X3.rvs(sample_size)-np.mean(X3.rvs(sample_size)))/np.std(X3.rvs(sample_size))
            X4 = get_truncated_normal(mean=3500-B1*250, sd=(1750-B1*125)*(1+A1), low=2500-B1*250, upp=4500-B1*250)
            np.random.seed(seed+A1+C1+B1+4)
            rnd4 = (X4.rvs(sample_size)-np.mean(X4.rvs(sample_size)))/np.std(X4.rvs(sample_size))

            rnd = np.concatenate((np.reshape(rnd1,(sample_size,1)), np.reshape(rnd2,(sample_size,1)), np.reshape(rnd3,(sample_size,1)), np.reshape(rnd4,(sample_size,1))), axis=1)

            #Correlations
            corr = 0.25*(C1+1)
            r1_2 = corr
            r1_3 = corr
            r1_4 = corr
            r2_3 = corr
            r2_4 = corr
            r3_4 = corr

            corr_mat = np.array([[1.0, r1_2, r1_3, r1_4],
                                [r1_2, 1.0, r2_3, r2_4],
                                [r1_3, r2_3, 1.0, r3_4],
                                [r1_4, r2_4, r3_4, 1.0]])

            #nearest positive semidefinite correlation matrix (if the chosen correlations do not lead to a semidefinite matrix)
            corr_mat1 = nearPD(corr_mat,nit=10)

            #Compute the (upper) Cholesky decomposition matrix
            upper_chol = cholesky(corr_mat1)
            ans = rnd @ upper_chol

            #reshape data
            np.random.seed(seed+A1+C1+B1+1)
            ans[:,0]=ans[:,0]*np.std(X1.rvs(sample_size))+np.mean(X1.rvs(sample_size))
            np.random.seed(seed+A1+C1+B1+2)
            ans[:,1]=ans[:,1]*np.std(X2.rvs(sample_size))+np.mean(X2.rvs(sample_size))
            np.random.seed(seed+A1+C1+B1+3)
            ans[:,2]=ans[:,2]*np.std(X3.rvs(sample_size))+np.mean(X3.rvs(sample_size))
            np.random.seed(seed+A1+C1+B1+4)
            ans[:,3]=ans[:,3]*np.std(X4.rvs(sample_size))+np.mean(X4.rvs(sample_size))

            #city 1
            if B1 == 0:
                lat = np.full((sample_size, 1), 47.37)
                long = np.full((sample_size, 1), 8.54)
                full = np.concatenate((np.rint(ans), lat, long), axis = 1)   

            #city 2                                
            if B1 == 1:
                lat = np.full((sample_size, 1), 46.20)
                long = np.full((sample_size, 1), 6.14)           
                full1 = np.concatenate((np.rint(ans), lat, long), axis = 1)

            #city 3                
            if B1 == 2:
                lat = np.full((sample_size, 1), 46.95)
                long = np.full((sample_size, 1), 7.44)
                full2 = np.concatenate((np.rint(ans), lat, long), axis = 1)

            #city 4                
            if B1 == 3:
                lat = np.full((sample_size, 1), 47.56)
                long = np.full((sample_size, 1), 7.59)                  
                full3 = np.concatenate((np.rint(ans), lat, long), axis = 1)                

                #putting all cities together
                dataset_unscaled = np.concatenate((full, full1, full2, full3), axis = 0)
                #scales data
                dataset = (dataset_unscaled-np.mean(dataset_unscaled, axis=0))/np.std(dataset_unscaled, axis=0)
                df = pd.DataFrame(data = dataset,
                            columns = ['roomMin', 'roomMax', 'priceMin', 'priceMax', 'lat', 'long'])


        #####Modifying the complete simulated dataset#####

        #D1 is for the missingness
        for D1 in range(0,3):

            #E1 is for the missingness design (0=MCAR, 1=MAR)
            for E1 in range(0,2):

                df_NaN_empty = pd.DataFrame()
                #F1 is for each city
                for F1 in range(0,4):                   

                    #percentage missing
                    #a for roomMin, b for roomMax, c for priceMin, d for priceMax
                    a = 0.05*(1+D1)*0.5+0.1*(-E1)**(F1+2)
                    b = 0.20*(1+D1)*0.5+0.15*(-E1)**(F1+1)
                    c = 0.30*(1+D1)*0.5+0.16*(-E1)**(F1+2)
                    d = 0.02*(1+D1)*0.5+0.05*(-E1)**(F1+1)

                    #create the 'missingness matrix'
                    np.random.seed(seed+A1+C1+B1+D1+E1+5)
                    df_NaN = pd.DataFrame({'NaN1' : np.random.sample(sample_size),
                                           'NaN2' : np.random.sample(sample_size),
                                           'NaN3' : np.random.sample(sample_size),
                                           'NaN4' : np.random.sample(sample_size),
                                           'NaN5' : np.full(sample_size,1),
                                           'NaN6' : np.full(sample_size,1)})

                    #transform the random values (uniform dist from 0 to 1) into integers referring to the missingness fraction
                    df_NaN.loc[(df_NaN.NaN1 <= a), 'NaN1'] = 0
                    df_NaN.loc[(df_NaN.NaN1 > a), 'NaN1'] = 1

                    df_NaN.loc[(df_NaN.NaN2 <= b), 'NaN2'] = 0
                    df_NaN.loc[(df_NaN.NaN2 > b), 'NaN2'] = 1

                    df_NaN.loc[(df_NaN.NaN3 <= c), 'NaN3'] = 0
                    df_NaN.loc[(df_NaN.NaN3 > c), 'NaN3'] = 1

                    df_NaN.loc[(df_NaN.NaN4 <= d), 'NaN4'] = 0
                    df_NaN.loc[(df_NaN.NaN4 > d), 'NaN4'] = 1

                    df_NaN_empty = df_NaN_empty.append(df_NaN, ignore_index = True)


                #multiply the missingness matrix with the dataset
                df_with_missing = pd.DataFrame(df.values*df_NaN_empty.values, columns=df.columns, index=df.index)

                #remove rows where roomMin and roomMax or priceMin and priceMax are missing
                Z = np.where((df_with_missing[df_with_missing.columns[0]] + df_with_missing[df_with_missing.columns[1]]==0) | (df_with_missing[df_with_missing.columns[2]] + df_with_missing[df_with_missing.columns[3]]==0))
                df_new = df_with_missing.drop(Z[0])

                #transform the 0 into NaN
                df_new.loc[(df_new.roomMin == 0),'roomMin']= np.nan
                df_new.loc[(df_new.roomMax == 0),'roomMax']= np.nan
                df_new.loc[(df_new.priceMin == 0),'priceMin']= np.nan
                df_new.loc[(df_new.priceMax == 0),'priceMax']= np.nan            


                #####Preparing data for performance testing and iterative imputation#####

                np.random.seed(seed+A1+C1+B1+D1+E1+6)
                #splitting the dataset into two subsets
                #df1 is for final performance testing (10% of the whole dataset) - the hold-out dataset
                #df2 is for training and testing (90% of the whole dataset)
                msk1 = np.random.rand(len(df_new)) < 0.1
                df1 = df_new[msk1]
                df2 = df_new[~msk1]

                #removing all rows with missing values in df2 (D0 in paper)
                df2_full = df2.dropna()
                np.random.seed(seed+A1+C1+B1+D1+E1+7)
                #splitting into test (df2_test, D0,2 in paper) (20%) and training (df2_train, D0,1 in paper) (80%) set
                msk2 = np.random.rand(len(df2_full)) < 0.8
                df2_train = df2_full[msk2]
                df2_test = df2_full[~msk2]
                np.random.seed(seed+A1+C1+B1+D1+E1+8)
                #second missingness matrix to determine which values from the test dataset (D0,2) to remove 
                #to have the same missingness pattern
                df_NaN2 = pd.DataFrame({'NaN1' : np.random.sample(len(df2_test)),
                                        'NaN2' : np.random.sample(len(df2_test)),
                                        'NaN3' : np.random.sample(len(df2_test)),
                                        'NaN4' : np.random.sample(len(df2_test)),
                                        'NaN5' : np.full(len(df2_test),1),
                                        'NaN6' : np.full(len(df2_test),1)})

                E1 = 0

                df_NaN2.loc[(df_NaN2.NaN1 <= a), 'NaN1'] = 0
                df_NaN2.loc[(df_NaN2.NaN1 > a), 'NaN1'] = 1

                df_NaN2.loc[(df_NaN2.NaN2 <= b), 'NaN2'] = 0
                df_NaN2.loc[(df_NaN2.NaN2 > b), 'NaN2'] = 1

                df_NaN2.loc[(df_NaN2.NaN3 <= c), 'NaN3'] = 0
                df_NaN2.loc[(df_NaN2.NaN3 > c), 'NaN3'] = 1

                df_NaN2.loc[(df_NaN2.NaN4 <= d), 'NaN4'] = 0
                df_NaN2.loc[(df_NaN2.NaN4 > d), 'NaN4'] = 1

                #multiply the missingness matrix with the dataset
                #df2_test_v1 (D0,2 mod in paper) is the test data set with the same missingness pattern as the original dataset (D in paper)
                df2_test_v1 = pd.DataFrame(df2_test.values*df_NaN2.values, columns=df2_test.columns, index=df2_test.index)


                #remove rows where roomMin and roomMax or priceMin and priceMax are missing and keeping the index
                Y = df2_test_v1[(df2_test_v1[df2_test_v1.columns[0]] + df2_test_v1[df2_test_v1.columns[1]]==0) | (df2_test_v1[df2_test_v1.columns[2]] + df2_test_v1[df2_test_v1.columns[3]]==0)]
                df2_test_new = df2_test_v1.drop(Y.index)

                #transform the 0 into NaN
                df2_test_new.loc[(df2_test_new.roomMin == 0),'roomMin']= np.nan
                df2_test_new.loc[(df2_test_new.roomMax == 0),'roomMax']= np.nan
                df2_test_new.loc[(df2_test_new.priceMin == 0),'priceMin']= np.nan
                df2_test_new.loc[(df2_test_new.priceMax == 0),'priceMax']= np.nan

                ##at the beginning, there are already complete rows in the test set
                ##those rows have to added to the train set   

                df2_test_new = df2_test_new[df2_test_new.isnull().sum(axis=1)!=0]
                df2_train = pd.concat([df2_train,df2_test_new[df2_test_new.isnull().sum(axis=1)==0]])
                df2_train_reset = pd.concat([df2_train,df2_test_new[df2_test_new.isnull().sum(axis=1)==0]])

                #combination of df2_train and df2_test_new (for later performance testing)
                df2_full1 = pd.concat([df2_train,df2_test_new])
                df2_full_comp = df2_full.loc[df2_full1.index]



C:\Users\richa\miniconda3\lib\site-packages\tensorflow-2.2.0rc4-py3.7-win-amd64.egg\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 9297 samples, validate on 1034 samples
Epoch 1/1000
9297/9297 [==============================] - 0s 13us/step - loss: 1.2951 - val_loss: 1.1461
Epoch 2/1000
9297/9297 [==============================] - 0s 1us/step - loss: 1.0063 - val_loss: 1.0636
Epoch 3/1000
9297/9297 [==============================] - 0s 1us/step - loss: 0.8484 - val_loss: 0.9868
Epoch 4/1000
9297/9297 [==============================] - 0s 1us/step - loss: 0.7035 - val_loss: 0.9162
Epoch 5/1000
9297/9297 [==============================] - 0s 1us/step - loss: 0.6003 - val_loss: 0.8447
Epoch 6/1000
9297/9297 [==============================] - 0s 1us/step - loss: 0.4945 - val_loss: 0.7705
Epoch 7/1000
9297/9297 [==============================] - 0s 1us/step - loss: 0.4012 - val_loss: 0.6975
Epoch 8/1000
9297/9297 [==============================] - 0s 1us/step - loss: 0.3163 - val_loss: 0.6252
Epoch 9/1000
9297/9297 [==============================] - 0s 1us/step - loss: 0.2411 - val_loss: 0.5677
Epoch 10/1000
9

C:\Users\richa\miniconda3\lib\site-packages\tensorflow-2.2.0rc4-py3.7-win-amd64.egg\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6226 samples, validate on 692 samples
Epoch 1/1000
6226/6226 [==============================] - 0s 21us/step - loss: 1.2080 - val_loss: 1.1232
Epoch 2/1000
6226/6226 [==============================] - 0s 1us/step - loss: 0.9639 - val_loss: 1.0372
Epoch 3/1000
6226/6226 [==============================] - 0s 1us/step - loss: 0.8010 - val_loss: 0.9721
Epoch 4/1000
6226/6226 [==============================] - 0s 1us/step - loss: 0.6924 - val_loss: 0.9122
Epoch 5/1000
6226/6226 [==============================] - 0s 1us/step - loss: 0.5897 - val_loss: 0.8335
Epoch 6/1000
6226/6226 [==============================] - 0s 1us/step - loss: 0.4887 - val_loss: 0.7604
Epoch 7/1000
6226/6226 [==============================] - 0s 1us/step - loss: 0.3961 - val_loss: 0.6837
Epoch 8/1000
6226/6226 [==============================] - 0s 1us/step - loss: 0.3172 - val_loss: 0.6301
Epoch 9/1000
6226/6226 [==============================] - 0s 1us/step - loss: 0.2579 - val_loss: 0.5753
Epoch 10/1000
62

C:\Users\richa\miniconda3\lib\site-packages\tensorflow-2.2.0rc4-py3.7-win-amd64.egg\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

Train on 6039 samples, validate on 671 samples
Epoch 1/1000
6039/6039 [==============================] - 0s 21us/step - loss: 1.4592 - val_loss: 1.4111
Epoch 2/1000
6039/6039 [==============================] - 0s 1us/step - loss: 1.1471 - val_loss: 1.3265
Epoch 3/1000
6039/6039 [==============================] - 0s 1us/step - loss: 1.0053 - val_loss: 1.2410
Epoch 4/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.8848 - val_loss: 1.1741
Epoch 5/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.7912 - val_loss: 1.1188
Epoch 6/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.7131 - val_loss: 1.0660
Epoch 7/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.6425 - val_loss: 1.0136
Epoch 8/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.5761 - val_loss: 0.9576
Epoch 9/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.5122 - val_loss: 0.9001
Epoch 10/1000
60

C:\Users\richa\miniconda3\lib\site-packages\tensorflow-2.2.0rc4-py3.7-win-amd64.egg\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 9041 samples, validate on 1005 samples
Epoch 1/1000
9041/9041 [==============================] - 0s 15us/step - loss: 1.9097 - val_loss: 1.5950
Epoch 2/1000
9041/9041 [==============================] - 0s 1us/step - loss: 1.4890 - val_loss: 1.4885
Epoch 3/1000
9041/9041 [==============================] - 0s 1us/step - loss: 1.2703 - val_loss: 1.3858
Epoch 4/1000
9041/9041 [==============================] - 0s 1us/step - loss: 1.0977 - val_loss: 1.3274
Epoch 5/1000
9041/9041 [==============================] - 0s 1us/step - loss: 0.9699 - val_loss: 1.2776
Epoch 6/1000
9041/9041 [==============================] - 0s 1us/step - loss: 0.8662 - val_loss: 1.2430
Epoch 7/1000
9041/9041 [==============================] - 0s 1us/step - loss: 0.7794 - val_loss: 1.2115
Epoch 8/1000
9041/9041 [==============================] - 0s 1us/step - loss: 0.7045 - val_loss: 1.1851
Epoch 9/1000
9041/9041 [==============================] - 0s 1us/step - loss: 0.6373 - val_loss: 1.1545
Epoch 10/1000
9

C:\Users\richa\miniconda3\lib\site-packages\tensorflow-2.2.0rc4-py3.7-win-amd64.egg\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6039 samples, validate on 671 samples
Epoch 1/1000
6039/6039 [==============================] - 0s 21us/step - loss: 1.3114 - val_loss: 1.2273
Epoch 2/1000
6039/6039 [==============================] - 0s 1us/step - loss: 1.0675 - val_loss: 1.0731
Epoch 3/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.9032 - val_loss: 1.0039
Epoch 4/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.8039 - val_loss: 0.9385
Epoch 5/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.7261 - val_loss: 0.8809
Epoch 6/1000
6039/6039 [==============================] - 0s 0us/step - loss: 0.6573 - val_loss: 0.8224
Epoch 7/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.5933 - val_loss: 0.7628
Epoch 8/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.5303 - val_loss: 0.6999
Epoch 9/1000
6039/6039 [==============================] - 0s 1us/step - loss: 0.4671 - val_loss: 0.6361
Epoch 10/1000
60

In [46]:
####Simulation of the dataset####
##procedure for chi squared distributed values##

seed = 1900
sample_size = 5000

#A1 is for the dispersion of the distribution (from low to high)
for A1 in range(0,3):   

    #C1 is for the correlation (from low to high)
    for C1 in range(0,3):   

        #for each city one separate sample
        for B1 in range(0,4):
            np.random.seed(seed+A1+C1+B1)
            rnd = np.random.chisquare(3, size=(sample_size*4,4))

            if C1 == 0:
            #Correlations
                r1_2 = 0.1
                r1_3 = 0.2
                r1_4 = 0.2
                r2_3 = 0.10
                r2_4 = 0.20
                r3_4 = 0.15

            if C1 == 1:
            #Correlations
                r1_2 = 0.55
                r1_3 = 0.5
                r1_4 = 0.55
                r2_3 = 0.45
                r2_4 = 0.55
                r3_4 = 0.45                  

            if C1 == 2:
            #Correlations
                r1_2 = 0.8
                r1_3 = 0.75
                r1_4 = 0.8
                r2_3 = 0.75
                r2_4 = 0.75
                r3_4 = 0.75


            corr_mat = np.array([[1.0, r1_2, r1_3, r1_4],
                                [r1_2, 1.0, r2_3, r2_4],
                                [r1_3, r2_3, 1.0, r3_4],
                                [r1_4, r2_4, r3_4, 1.0]])

            #positive semidefinite correlation matrix
            corr_mat1 = nearPD(corr_mat,nit=10)

            #Compute the (upper) Cholesky decomposition matrix
            upper_chol = cholesky(corr_mat1)
            ans = rnd @ upper_chol

            #first line scales the standard deviation
            #second line scales the mean

            ans[:,0] = (ans[:,0]/math.sqrt(6))*1*(A1+1)
            ans[:,0] = ans[:,0]+(2-np.mean(ans[:,0]))

            ans[:,1] = (ans[:,1]/math.sqrt(6))*2*(A1+1)
            ans[:,1] = ans[:,1]+(4-np.mean(ans[:,1]))

            ans[:,2] = (ans[:,2]/math.sqrt(6))*(750-B1*62.5)*(A1+1)
            ans[:,2] = ans[:,2]+(1500-B1*125-np.mean(ans[:,2]))

            ans[:,3] = (ans[:,3]/math.sqrt(6))*(1750-B1*125)*(A1+1)
            ans[:,3] = ans[:,3]+(3500-B1*250-np.mean(ans[:,3]))

            #removing rows where roomMin < 0.5
            A2 = ans[np.where(ans[:,0]>= 0.5)]
            #removing rows where roomMin > roomMax
            A3 = A2[np.where(A2[:,0] <= A2[:,1])]
            #removing rows where priceMin > priceMax
            A4 = A3[np.where(A3[:,2] <= A3[:,3])]


            #city 1
            if B1 == 0:
                lat = np.full((sample_size, 1), 47.37)
                long = np.full((sample_size, 1), 8.54)
                full = np.concatenate((np.rint(A4[:sample_size]), lat, long), axis = 1)   

            #city 2                                
            if B1 == 1:
                lat = np.full((sample_size, 1), 46.20)
                long = np.full((sample_size, 1), 6.14)           
                full1 = np.concatenate((np.rint(A4[:sample_size]), lat, long), axis = 1)

            #city 3                
            if B1 == 2:
                lat = np.full((sample_size, 1), 46.95)
                long = np.full((sample_size, 1), 7.44)
                full2 = np.concatenate((np.rint(A4[:sample_size]), lat, long), axis = 1)

            #city 4                
            if B1 == 3:
                lat = np.full((sample_size, 1), 47.56)
                long = np.full((sample_size, 1), 7.59)                  
                full3 = np.concatenate((np.rint(A4[:sample_size]), lat, long), axis = 1)                

                #putting all cities together
                dataset_unscaled = np.concatenate((full, full1, full2, full3), axis = 0)
                #scales data
                dataset = (dataset_unscaled-np.mean(dataset_unscaled, axis=0))/np.std(dataset_unscaled, axis=0)
                df = pd.DataFrame(data = dataset,
                            columns = ['roomMin', 'roomMax', 'priceMin', 'priceMax', 'lat', 'long'])



        #####Modifying the complete simulated dataset#####

        #D1 is for the missingness (low to high)
        for D1 in range(0,3):

            #E1 is for the missingness design (0=MCAR, 1=MAR)
            for E1 in range(0,2):

                df_NaN_empty = pd.DataFrame()
                #F1 is for each city
                for F1 in range(0,4):                   

                    #percentage missing
                    #a for roomMin, b for roomMax, c for priceMin, d for priceMax
                    a = 0.05*(1+D1)*0.5+0.1*(-E1)**(F1+2)
                    b = 0.20*(1+D1)*0.5+0.15*(-E1)**(F1+1)
                    c = 0.30*(1+D1)*0.5+0.16*(-E1)**(F1+2)
                    d = 0.02*(1+D1)*0.5+0.05*(-E1)**(F1+1)

                    np.random.seed(seed+A1+C1+B1+D1+E1+1)
                    #create the 'missingness matrix'
                    df_NaN = pd.DataFrame({'NaN1' : np.random.sample(sample_size),
                                           'NaN2' : np.random.sample(sample_size),
                                           'NaN3' : np.random.sample(sample_size),
                                           'NaN4' : np.random.sample(sample_size),
                                           'NaN5' : np.full(sample_size,1),
                                           'NaN6' : np.full(sample_size,1)})

                    #transform the random values (uniform dist from 0 to 1) into integers referring to the missingness fraction
                    df_NaN.loc[(df_NaN.NaN1 <= a), 'NaN1'] = 0
                    df_NaN.loc[(df_NaN.NaN1 > a), 'NaN1'] = 1

                    df_NaN.loc[(df_NaN.NaN2 <= b), 'NaN2'] = 0
                    df_NaN.loc[(df_NaN.NaN2 > b), 'NaN2'] = 1

                    df_NaN.loc[(df_NaN.NaN3 <= c), 'NaN3'] = 0
                    df_NaN.loc[(df_NaN.NaN3 > c), 'NaN3'] = 1

                    df_NaN.loc[(df_NaN.NaN4 <= d), 'NaN4'] = 0
                    df_NaN.loc[(df_NaN.NaN4 > d), 'NaN4'] = 1

                    df_NaN_empty = df_NaN_empty.append(df_NaN, ignore_index = True)


                #multiply the missingness matrix with the dataset
                df_with_missing = pd.DataFrame(df.values*df_NaN_empty.values, columns=df.columns, index=df.index)

                #remove rows where roomMin and roomMax or priceMin and priceMax are missing
                Z = np.where((df_with_missing[df_with_missing.columns[0]] + df_with_missing[df_with_missing.columns[1]]==0) | (df_with_missing[df_with_missing.columns[2]] + df_with_missing[df_with_missing.columns[3]]==0))
                df_new = df_with_missing.drop(Z[0])

                #transform the 0 into NaN
                df_new.loc[(df_new.roomMin == 0),'roomMin']= np.nan
                df_new.loc[(df_new.roomMax == 0),'roomMax']= np.nan
                df_new.loc[(df_new.priceMin == 0),'priceMin']= np.nan
                df_new.loc[(df_new.priceMax == 0),'priceMax']= np.nan            


                #####Preparing data for performance testing and iterative imputation#####

                np.random.seed(seed+A1+C1+B1+D1+E1+2)
                #splitting the dataset into two subsets
                #df1 is for final performance testing (10% of the whole dataset) - the hold-out dataset
                #df2 is for training and testing (90% of the whole dataset)
                msk1 = np.random.rand(len(df_new)) < 0.1
                df1 = df_new[msk1]
                df2 = df_new[~msk1]

                #removing all rows with missing values in df2 (D0 in paper)
                df2_full = df2.dropna()
                np.random.seed(seed+A1+C1+B1+D1+E1+3)
                #splitting into test (df2_test, D0,2 in paper) (20%) and training (df2_train, D0,1 in paper) (80%) set
                msk2 = np.random.rand(len(df2_full)) < 0.8
                df2_train = df2_full[msk2]
                df2_test = df2_full[~msk2]
                np.random.seed(seed+A1+C1+B1+D1+E1+4)
                #second missingness matrix to determine which values from the test dataset (D0,2) to remove 
                #to have the same missingness pattern
                df_NaN2 = pd.DataFrame({'NaN1' : np.random.sample(len(df2_test)),
                                        'NaN2' : np.random.sample(len(df2_test)),
                                        'NaN3' : np.random.sample(len(df2_test)),
                                        'NaN4' : np.random.sample(len(df2_test)),
                                        'NaN5' : np.full(len(df2_test),1),
                                        'NaN6' : np.full(len(df2_test),1)})

                E1 = 0

                df_NaN2.loc[(df_NaN2.NaN1 <= a), 'NaN1'] = 0
                df_NaN2.loc[(df_NaN2.NaN1 > a), 'NaN1'] = 1

                df_NaN2.loc[(df_NaN2.NaN2 <= b), 'NaN2'] = 0
                df_NaN2.loc[(df_NaN2.NaN2 > b), 'NaN2'] = 1

                df_NaN2.loc[(df_NaN2.NaN3 <= c), 'NaN3'] = 0
                df_NaN2.loc[(df_NaN2.NaN3 > c), 'NaN3'] = 1

                df_NaN2.loc[(df_NaN2.NaN4 <= d), 'NaN4'] = 0
                df_NaN2.loc[(df_NaN2.NaN4 > d), 'NaN4'] = 1

                #multiply the missingness matrix with the dataset
                #df2_test_v1 (D0,2 mod in paper) is the test data set with the same missingness pattern as the original dataset (D in paper)
                df2_test_v1 = pd.DataFrame(df2_test.values*df_NaN2.values, columns=df2_test.columns, index=df2_test.index)


                #remove rows where roomMin and roomMax or priceMin and priceMax are missing and keeping the index
                Y = df2_test_v1[(df2_test_v1[df2_test_v1.columns[0]] + df2_test_v1[df2_test_v1.columns[1]]==0) | (df2_test_v1[df2_test_v1.columns[2]] + df2_test_v1[df2_test_v1.columns[3]]==0)]
                df2_test_new = df2_test_v1.drop(Y.index)

                #transform the 0 into NaN
                df2_test_new.loc[(df2_test_new.roomMin == 0),'roomMin']= np.nan
                df2_test_new.loc[(df2_test_new.roomMax == 0),'roomMax']= np.nan
                df2_test_new.loc[(df2_test_new.priceMin == 0),'priceMin']= np.nan
                df2_test_new.loc[(df2_test_new.priceMax == 0),'priceMax']= np.nan

                ##at the beginning, there are already complete rows in the test set
                ##those rows have to added to the train set   

                df2_test_new = df2_test_new[df2_test_new.isnull().sum(axis=1)!=0]
                df2_train = pd.concat([df2_train,df2_test_new[df2_test_new.isnull().sum(axis=1)==0]])
                df2_train_reset = pd.concat([df2_train,df2_test_new[df2_test_new.isnull().sum(axis=1)==0]])

                #combination of df2_train and df2_test_new (for later performance testing)
                df2_full1 = pd.concat([df2_train,df2_test_new])
                df2_full_comp = df2_full.loc[df2_full1.index]



C:\Users\richa\miniconda3\lib\site-packages\tensorflow-2.2.0rc4-py3.7-win-amd64.egg\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 9997 samples, validate on 1111 samples
Epoch 1/1000
9997/9997 [==============================] - 0s 13us/step - loss: 1.2851 - val_loss: 1.1481
Epoch 2/1000
9997/9997 [==============================] - 0s 1us/step - loss: 1.0277 - val_loss: 1.1046
Epoch 3/1000
9997/9997 [==============================] - 0s 1us/step - loss: 0.9203 - val_loss: 1.0657
Epoch 4/1000
9997/9997 [==============================] - 0s 1us/step - loss: 0.8338 - val_loss: 1.0537
Epoch 5/1000
9997/9997 [==============================] - 0s 1us/step - loss: 0.7740 - val_loss: 1.0396
Epoch 6/1000
9997/9997 [==============================] - 0s 1us/step - loss: 0.7253 - val_loss: 1.0355
Epoch 7/1000
9997/9997 [==============================] - 0s 1us/step - loss: 0.6803 - val_loss: 1.0218
Epoch 8/1000
9997/9997 [==============================] - 0s 1us/step - loss: 0.6343 - val_loss: 1.0158
Epoch 9/1000
9997/9997 [==============================] - 0s 1us/step - loss: 0.5856 - val_loss: 0.9998
Epoch 10/1000
9

C:\Users\richa\miniconda3\lib\site-packages\tensorflow-2.2.0rc4-py3.7-win-amd64.egg\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8058 samples, validate on 896 samples
Epoch 1/1000
8058/8058 [==============================] - 0s 17us/step - loss: 1.2581 - val_loss: 1.1427
Epoch 2/1000
8058/8058 [==============================] - 0s 1us/step - loss: 1.0402 - val_loss: 1.1238
Epoch 3/1000
8058/8058 [==============================] - 0s 1us/step - loss: 0.9274 - val_loss: 1.0847
Epoch 4/1000
8058/8058 [==============================] - 0s 1us/step - loss: 0.8379 - val_loss: 1.0785
Epoch 5/1000
8058/8058 [==============================] - 0s 1us/step - loss: 0.7743 - val_loss: 1.0680
Epoch 6/1000
8058/8058 [==============================] - 0s 1us/step - loss: 0.7190 - val_loss: 1.0651
Epoch 7/1000
8058/8058 [==============================] - 0s 1us/step - loss: 0.6664 - val_loss: 1.0544
Epoch 8/1000
8058/8058 [==============================] - 0s 1us/step - loss: 0.6119 - val_loss: 1.0496
Epoch 9/1000
8058/8058 [==============================] - 0s 1us/step - loss: 0.5550 - val_loss: 1.0381
Epoch 10/1000
80

C:\Users\richa\miniconda3\lib\site-packages\tensorflow-2.2.0rc4-py3.7-win-amd64.egg\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


...

Train on 7943 samples, validate on 883 samples
Epoch 1/1000
7943/7943 [==============================] - 0s 16us/step - loss: 1.3508 - val_loss: 1.3435
Epoch 2/1000
7943/7943 [==============================] - 0s 1us/step - loss: 1.0275 - val_loss: 1.1945
Epoch 3/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.8626 - val_loss: 1.1363
Epoch 4/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.7714 - val_loss: 1.0858
Epoch 5/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.6998 - val_loss: 1.0417
Epoch 6/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.6362 - val_loss: 1.0015
Epoch 7/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.5776 - val_loss: 0.9659
Epoch 8/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.5230 - val_loss: 0.9359
Epoch 9/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.4727 - val_loss: 0.9127
Epoch 10/1000
79

C:\Users\richa\miniconda3\lib\site-packages\tensorflow-2.2.0rc4-py3.7-win-amd64.egg\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 9846 samples, validate on 1094 samples
Epoch 1/1000
9846/9846 [==============================] - 0s 13us/step - loss: 1.2718 - val_loss: 1.0722
Epoch 2/1000
9846/9846 [==============================] - 0s 1us/step - loss: 1.0398 - val_loss: 1.0091
Epoch 3/1000
9846/9846 [==============================] - 0s 1us/step - loss: 0.9103 - val_loss: 0.9659
Epoch 4/1000
9846/9846 [==============================] - 0s 1us/step - loss: 0.8051 - val_loss: 0.9321
Epoch 5/1000
9846/9846 [==============================] - 0s 1us/step - loss: 0.7229 - val_loss: 0.9020
Epoch 6/1000
9846/9846 [==============================] - 0s 1us/step - loss: 0.6490 - val_loss: 0.8763
Epoch 7/1000
9846/9846 [==============================] - 0s 1us/step - loss: 0.5831 - val_loss: 0.8530
Epoch 8/1000
9846/9846 [==============================] - 0s 1us/step - loss: 0.5258 - val_loss: 0.8361
Epoch 9/1000
9846/9846 [==============================] - 0s 1us/step - loss: 0.4778 - val_loss: 0.8232
Epoch 10/1000
9

C:\Users\richa\miniconda3\lib\site-packages\tensorflow-2.2.0rc4-py3.7-win-amd64.egg\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 7943 samples, validate on 883 samples
Epoch 1/1000
7943/7943 [==============================] - 0s 16us/step - loss: 1.3425 - val_loss: 1.2587
Epoch 2/1000
7943/7943 [==============================] - 0s 1us/step - loss: 1.0608 - val_loss: 1.1794
Epoch 3/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.9307 - val_loss: 1.1259
Epoch 4/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.8185 - val_loss: 1.0706
Epoch 5/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.7355 - val_loss: 1.0395
Epoch 6/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.6656 - val_loss: 1.0034
Epoch 7/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.6037 - val_loss: 0.9830
Epoch 8/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.5490 - val_loss: 0.9615
Epoch 9/1000
7943/7943 [==============================] - 0s 1us/step - loss: 0.5015 - val_loss: 0.9544
Epoch 10/1000
79